In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
#Probably don't need this function unless we decide to also join on position in case there are conflicting names
#Takes in a dataframe, a position column name ("Pos" for stats df, "position" for salary df), 
#and index to insert new column into (4 for stats df, 5 for salary df).
#So call to a stats df should be posBigOrSmall(stats_df, "Pos", 4) 
#and for salary df posBigOrSmall(salary_df, "position", 5)
def posBigOrSmall(df, pos_colname, col_index):
    pos = []
    for i in range(len(df)):
        if df[pos_colname][i] == "PG" or df[pos_colname][i] == "SG":
            pos.append("small")
        else:
            pos.append("big")
    df.insert(col_index, 'pos', pos)

In [3]:
#Takes in a stats dataframe and a salary dataframe, then cleans up the 
#names column in both and joins them by name. Returns the joined dataframe.

def joinTables(stats_df, salary_df): 
    stats_df = stats_df.drop(['Unnamed: 19','Unnamed: 24'], axis = 1)
    stats_df.rename(columns = {'Tm': 'team'}, inplace = True)
    
    #Add 'name' column to the stats_df
    name_temp = stats_df['Player'].map(lambda x: 
                                        re.sub("[^a-zA-Z]+", "", 
                                               re.search("^.*(?=\\\\)", x).group(0).replace(" ", "").lower()))
    stats_df.insert(1, 'name', name_temp.map(lambda x: re.sub("[^a-zA-Z]+", "", x)))
    #Add 'pos' column to the stats_df
    posBigOrSmall(stats_df, "Pos", 4) 
    
    #Add 'name' column to salary_df
    name_col = (salary_df['first name'] + salary_df['last name']).str.lower()
    for i in range(len(name_col)):
        new_name = re.sub("[^a-zA-Z]+", "", name_col[i])
        if new_name.endswith('jr'):
            new_name = new_name[:-2]
        name_col[i] = new_name
    salary_df.insert(0, 'name', name_col)
    #Add 'pos' column to the salary_df
    posBigOrSmall(salary_df, "position", 5)
    
    combined_table = stats_df.merge(salary_df, how = 'inner', on = ['name'])
    
    #Print out which names are in the salary df but not included in the combined df
    combined_names = combined_table['name'].tolist()
    salary_names = salary_df['name'].tolist()
    differing_names = []
    for i in range((len(salary_names))):
        if salary_names[i] not in combined_names:
            differing_names.append(salary_names[i])
    print(differing_names)
    print('salary_table length: ' + str(len(salary_df)))
    print('combined_table length: ' + str(len(combined_table)))
    return combined_table

In [4]:
adv_stats2017 = pd.read_csv('player-data/player2017/Adv-2017.csv')
fa_salary2017 = pd.read_csv('salary-data/fa-signings/2017.csv')
combined_fa2017 = joinTables(adv_stats2017, fa_salary2017)
combined_fa2017.head()

['patrickmills', 'lucrichard']
salary_table length: 110
combined_table length: 150


,Rk,name,Player,Pos,pos_x,Age,team_x,G,MP,PER,...,VORP,last name,first name,age,position,pos_y,team_y,faStatus,total-contract,average
0,4,arronafflalo,Arron Afflalo\afflaar01,SG,small,31,SAC,61,1580,8.9,...,-0.6,Afflalo,Arron,31,SG,small,SAC,UFA,2328652,2328652
1,9,tonyallen,Tony Allen\allento01,SG,small,35,MEM,71,1914,13.3,...,1.3,Allen,Tony,35,SG,small,MEM,UFA,2328652,2328652
2,23,lukebabbitt,Luke Babbitt\babbilu01,SF,big,27,MIA,68,1065,8.3,...,0.0,Babbitt,Luke,28,SF,big,MIA,UFA,1974159,1974159
3,24,ronbaker,Ron Baker\bakerro01,SG,small,23,NYK,52,857,7.5,...,-0.5,Baker,Ron,24,SG,small,NYK,UFA,8872400,4436200
4,34,aronbaynes,Aron Baynes\baynear01,C,big,30,DET,75,1163,13.1,...,0.1,Baynes,Aron,30,PF,big,DET,UFA,4328000,4328000


In [5]:
adv_stats2016 = pd.read_csv('player-data/player2016/Adv-2016.csv')
fa_salary2016 = pd.read_csv('salary-data/fa-signings/2016.csv')
combined_fa2016 = joinTables(adv_stats2016, fa_salary2016)
combined_fa2016.head()

['ishmaelsmith', 'justinhamilton', 'lucrichard', 'louisamundson', 'jamesmcadoo']
salary_table length: 130
combined_table length: 161


,Rk,name,Player,Pos,pos_x,Age,team_x,G,MP,PER,...,VORP,last name,first name,age,position,pos_y,team_y,faStatus,total-contract,average
0,1,quincyacy,Quincy Acy\acyqu01,PF,big,25,SAC,59,876,14.7,...,0.5,Acy,Quincy,26,SF,big,SAC,UFA,2229953,1114977
1,4,arronafflalo,Arron Afflalo\afflaar01,SG,small,30,NYK,71,2371,10.9,...,-0.2,Afflalo,Arron,31,SG,small,NYK,UFA,25000000,12500000
2,13,chrisandersen,Chris Andersen\anderch01,C,big,37,TOT,27,402,15.3,...,0.3,Andersen,Chris,39,PF,big,MEM,UFA,1551659,1551659
3,13,chrisandersen,Chris Andersen\anderch01,C,big,37,MIA,7,36,17.6,...,0.0,Andersen,Chris,39,PF,big,MEM,UFA,1551659,1551659
4,13,chrisandersen,Chris Andersen\anderch01,C,big,37,MEM,20,366,15.1,...,0.2,Andersen,Chris,39,PF,big,MEM,UFA,1551659,1551659
